## Data Cleaning & Transformation
- This module will focus on real wworld data engineering practices for cleaning and transforming data at scale using pyspark

## Steps
**identify issue**--missing values,duplivcate,invalid data

**handle missing value**--drop,fill,impute missing values

**standardize formate**--normalize categorical fields,convert date time

**data type correction**--correct data type is there for each column

**Duplication**--remove duplicate records

**Data transformation**--apply feature engineering

**Store cleaned data**--data in hdfs,.parquet format

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder \
.appName('OlistData') \
.getOrCreate()

25/04/25 06:07:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
order_item_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path +'product_category_name_translation.csv',header=True,inferSchema=True)

In [1]:
hdfs_path='/data/olist/'

In [3]:
customers_df.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [5]:
## Identify Missing values

In [6]:
from pyspark.sql.functions import *

In [9]:
def  missing_values (df,df_name):
    print(f'Missing values in{df_name}:')
    df.select([count(when(col(c).isNull(),1)).alias(c) for c in df.columns]).show()

In [10]:
missing_values(customers_df,'customer')

Missing values incustomer:
+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [11]:
missing_values(orders_df,'orders')

Missing values inorders:


+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [12]:
missing_values(order_item_df,'order_item')

Missing values inorder_item:
+--------+-------------+----------+---------+-------------------+-----+-------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|
+--------+-------------+----------+---------+-------------------+-----+-------------+
|       0|            0|         0|        0|                  0|    0|            0|
+--------+-------------+----------+---------+-------------------+-----+-------------+



In [13]:
missing_values(payments_df,'payments')

Missing values inpayments:
+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+



## Hndle Missing Values
 1 Drop Missing Values(for non-critical columns)

2 Fill missing values(for numerical columns)

3 Impute Missing values(for continuous data)

In [14]:
orders_df_cleaned =orders_df.na.drop(subset=['order_id','customer_id','order_status'])

In [15]:
orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [17]:
orders_df_cleaned=orders_df.fillna({'order_delivered_customer_date':'9999-12-21'})

In [18]:
orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

## Impute Missing Values

In [22]:
from pyspark.ml.feature import Imputer 
imputer=Imputer(inputCols=['payment_value'],outputCols=['payments_value_imputes']).setStrategy('mean')
payments_df_cleaned = imputer.fit(payments_df).transform(payments_df)

In [23]:
payments_df_cleaned.show()

+--------------------+------------------+------------+--------------------+-------------+----------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payments_value_imputes|
+--------------------+------------------+------------+--------------------+-------------+----------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|                 99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                 24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                 65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|                107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|                128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|       

In [25]:
payments_df_with_null = payments_df.withColumn('payment_value',when(col('payment_value')!= 99.33,col('payment_value')).otherwise(lit(None)))



In [26]:
payments_df_with_null.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [27]:
from pyspark.ml.feature import Imputer 
imputer=Imputer(inputCols=['payment_value'],outputCols=['payments_value_imputes']).setStrategy('median')
payments_df_cleaned = imputer.fit(payments_df_with_null).transform(payments_df_with_null)


In [28]:
payments_df_cleaned.show()

+--------------------+------------------+------------+--------------------+-------------+----------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payments_value_imputes|
+--------------------+------------------+------------+--------------------+-------------+----------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|                 100.0|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                 24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                 65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|                107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|                128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|       

## Standardizing the format

In [30]:
def print_schema(df,df_name):
    print(f'schema of{df_name}:')
    df.printSchema()

In [31]:
print_schema(orders_df,'orders')

schema oforders:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [32]:
print_schema(customers_df,'customers')

schema ofcustomers:
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [33]:
print_schema(payments_df,'payments')

schema ofpayments:
root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [34]:
orders_df.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [36]:
orders_df_cleaned=orders_df_cleaned.withColumn('order_purchase_timestamp',to_date(col('order_purchase_timestamp')))
                                               

In [37]:
orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|              2017-10-02|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|              2018-07-24|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [40]:
payments_df_cleaned=payments_df_cleaned.withColumn('payments_type',when(col('payment_type')=='boleto','Bank Transfer')
                                                   .when(col('payment_type')=='credit_card','Credit Card')
                                                   .when(col('payment_type')== 'debit_card','Debit Card')
                                                   .otherwise('other'))

In [41]:
payments_df_cleaned.show()

+--------------------+------------------+------------+--------------------+-------------+----------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payments_value_imputes|payments_type|
+--------------------+------------------+------------+--------------------+-------------+----------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|                 100.0|  Credit Card|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                 24.39|  Credit Card|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                 65.71|  Credit Card|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|                107.78|  Credit Card|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|                128.45

In [42]:
 customers_df.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [44]:
customers_df_cleaned=customers_df.withColumn('customer_zip_code_prefix',col('customer_zip_code_prefix').cast('string'))

In [45]:
customers_df_cleaned.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

## Remove Duplicate Recoreds

In [48]:
customers_df_cleaned=customers_df_cleaned.dropDuplicates(['customer_id'])

### Data Transformation

In [54]:
order_with_details = orders_df_cleaned.join(order_item_df,'order_id','left')\
.join(payments_df_cleaned,'order_id','left')\
.join(customers_df_cleaned,'customer_id','left')

In [56]:
order_with_details.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+------+-------------+------------------+------------+--------------------+-------------+----------------------+-------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|payment_sequential|payment_type|payment_installments|payment_value|payments_value_imputes|payments_type|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+---------

In [58]:
order_with_total_value=order_with_details.groupBy('order_id')\
.agg(sum('payment_value').alias('total_order_value'))

## Adavance Transformation

In [61]:
order_item_df.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [62]:
order_item_df.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            112650|
|   mean|120.65373901471354|
| stddev|183.63392805026012|
|    min|              0.85|
|    25%|              39.9|
|    50%|             74.99|
|    75%|             134.9|
|    max|            6735.0|
+-------+------------------+



In [65]:
quantiles = order_item_df.approxQuantile('price',[0.01,0.99],0.0)
low_cutoff,high_cutoff = quantiles[0],quantiles[1]

In [66]:
low_cutoff,high_cutoff

(9.99, 890.0)

In [67]:
order_item_cleaned=order_item_df.filter((col('price')>=low_cutoff) & (col('price')<= high_cutoff))

In [69]:
payments_df_cleaned.select('payment_installments').summary().show()

+-------+--------------------+
|summary|payment_installments|
+-------+--------------------+
|  count|              103886|
|   mean|   2.853348863176944|
| stddev|  2.6870506738564925|
|    min|                   0|
|    25%|                   1|
|    50%|                   1|
|    75%|                   4|
|    max|                  24|
+-------+--------------------+



In [70]:
products_df.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [72]:
products_df_cleaned = products_df.withColumn(
'product_size_category',
when(col('product_weight_g') <500,'Small')
.when(col('product_weight_g').between(500,2000),'Medium')
.otherwise('Large')
)

In [73]:
products_df_cleaned.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_size_category|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|                Small|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|        

In [75]:
!hadoop fs -mkdir /data/olist_proc

In [76]:
order_with_details.write.mode('overwrite').parquet('/data/olist_proc/cleaned_data.parquent')

In [77]:
products_df_cleaned.write.mode('overwrite').parquet('/data/olist_proc/product_data.parquent')

In [78]:
!hadoop fs -ls /data/olist_proc

Found 2 items
drwxr-xr-x   - root hadoop          0 2025-04-25 07:36 /data/olist_proc/cleaned_data.parquent
drwxr-xr-x   - root hadoop          0 2025-04-25 07:37 /data/olist_proc/product_data.parquent


In [79]:
order_with_details.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: date (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- payments_value_imputes: double (nullable = true)
 |-- payments_type: string (nullable = true)
 |-- customer_